In [1]:
import qutip as qt
import numpy as np
print(qt.__version__)


5.0.4


## First we define state and density matrix

In [2]:

def create_random_state(n):
    basis_states = [qt.basis([2]*n, [int(i) for i in format(j, f'0{n}b')]) for j in range(2**n)]
    
    random_probs = np.random.rand(2**n)
    random_probs /= np.sum(random_probs)  # Normalize to sum to 1
    
    coefficients = np.sqrt(random_probs)
   
    random_state = sum(coeff * basis for coeff, basis in zip(coefficients, basis_states))
    return random_state

def compute_density_matrix(state):
    return state * state.dag()

num_qubits = 3  

random_state = create_random_state(num_qubits)

p_random = compute_density_matrix(random_state)


print("Random state:")
print(random_state)
np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("Density Matrix in Matrix Form:")
print(np.real(p_random.full()))



Random state:
Quantum object: dims=[[2, 2, 2], [1, 1, 1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[0.15776834]
 [0.39442983]
 [0.30627739]
 [0.41989573]
 [0.43354136]
 [0.18587238]
 [0.41011058]
 [0.3983951 ]]
Density Matrix in Matrix Form:
[[0.025 0.062 0.048 0.066 0.068 0.029 0.065 0.063]
 [0.062 0.156 0.121 0.166 0.171 0.073 0.162 0.157]
 [0.048 0.121 0.094 0.129 0.133 0.057 0.126 0.122]
 [0.066 0.166 0.129 0.176 0.182 0.078 0.172 0.167]
 [0.068 0.171 0.133 0.182 0.188 0.081 0.178 0.173]
 [0.029 0.073 0.057 0.078 0.081 0.035 0.076 0.074]
 [0.065 0.162 0.126 0.172 0.178 0.076 0.168 0.163]
 [0.063 0.157 0.122 0.167 0.173 0.074 0.163 0.159]]


## Conditional Mutual Information bound check

In [3]:
def compute_reduced_density_matrices(p_W):
    p_A = p_random.ptrace([0])  # Tracing out B and C
    p_B = p_random.ptrace([1])  # Tracing out A and C
    p_C = p_random.ptrace([2])  # Tracing out A and B
    p_AB = p_random.ptrace([0, 1])  # Tracing out C
    p_AC = p_random.ptrace([0, 2])  # Tracing out B
    p_BC = p_random.ptrace([1, 2])  # Tracing out A
    return p_A, p_B, p_C, p_AB, p_AC, p_BC


p_A, p_B, p_C, p_AB, p_AC, p_BC = compute_reduced_density_matrices(p_random)

def compute_entropies(p_A, p_B, p_C, p_AB, p_AC, p_BC, p_W):
    S_A = qt.entropy_vn(p_A)
    S_B = qt.entropy_vn(p_B)
    S_C = qt.entropy_vn(p_C)
    S_AB = qt.entropy_vn(p_AB)
    S_AC = qt.entropy_vn(p_AC)
    S_BC = qt.entropy_vn(p_BC)
    S_W = qt.entropy_vn(p_W)  # Total system entropy
    return S_A, S_B, S_C, S_AB, S_AC, S_BC, S_W

S_A, S_B, S_C, S_AB, S_AC, S_BC, S_W = compute_entropies(p_A, p_B, p_C, p_AB, p_AC, p_BC, p_random)


In [4]:
# Nicely formatted printout
entropy_labels = ["S(ρ_A)", "S(ρ_B)", "S(ρ_C)", "S(ρ_AB)", "S(ρ_AC)", "S(ρ_BC)", "S(ρ_W)"]
entropy_values = [S_A, S_B, S_C, S_AB, S_AC, S_BC, S_W]

# Print header
print("=" * 40)
print(f"{'Subsystem':<10} | {'Entropy':>12}")
print("=" * 40)

# Print entropy values
for label, value in zip(entropy_labels, entropy_values):
    print(f"{label:<10} | {value:12.6f}")

print("=" * 40)


Subsystem  |      Entropy
S(ρ_A)     |     0.234049
S(ρ_B)     |     0.114002
S(ρ_C)     |     0.239095
S(ρ_AB)    |     0.239095
S(ρ_AC)    |     0.114002
S(ρ_BC)    |     0.234049
S(ρ_W)     |     0.000000


## Compute the Conditional Mutual Information

In [5]:
I_ABcC = S_AC + S_BC - S_W - S_C
print("Conditional mutual information I(A:B|C) =",I_ABcC)

Conditional mutual information I(A:B|C) = 0.10895603272771079


## Compute 3 possible bounds using Z operator

In [6]:
#Z = (qt.sigmaz() + qt.sigmax())/(np.sqrt(2))
Z = qt.sigmaz()
Z_2 = qt.tensor(Z, Z)
Z_full = qt.tensor(Z, Z, Z)


avg_Z_A = (p_A * Z).tr()
avg_Z_B = (p_B * Z).tr()
avg_Z_C = (p_C * Z).tr()
avg_Z_AC = (p_AC * Z_2).tr()
avg_Z_BC = (p_BC * Z_2).tr()
avg_Z_AB = (p_AB * Z_2).tr()
avg_Z_full = (p_random * Z_full).tr()

Z_norm = 1

np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("Density Matrix in Matrix Form:")
print(np.real(Z_full.full()))


Density Matrix in Matrix Form:
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.]]


In [7]:
# Define labels and values
z_labels = [
    "⟨Z_A⟩", "⟨Z_B⟩", "⟨Z_C⟩",
    "⟨Z_AC⟩", "⟨Z_BC⟩", "⟨Z_AB⟩",
    "⟨Z_W⟩ (Full)"
]
z_values = [
    avg_Z_A, avg_Z_B, avg_Z_C,
    avg_Z_AC, avg_Z_BC, avg_Z_AB,
    avg_Z_full
]

# Print header
print("=" * 40)
print(f"{'Observable':<10} | {'Expectation Value':>20}")
print("=" * 40)

# Print expectation values
for label, value in zip(z_labels, z_values):
    print(f"{label:<10} | {value:20.6f}")

print("=" * 40)


Observable |    Expectation Value
⟨Z_A⟩      |  -0.098832+0.000000j
⟨Z_B⟩      |  -0.194055+0.000000j
⟨Z_C⟩      |  -0.050309+0.000000j
⟨Z_AC⟩     |  -0.376072+0.000000j
⟨Z_BC⟩     |   0.095760+0.000000j
⟨Z_AB⟩     |   0.014750+0.000000j
⟨Z_W⟩ (Full) |  -0.192115+0.000000j


In [15]:
Var_A = 1 - avg_Z_A**2
Var_B = 1 - avg_Z_B**2
C_AB = avg_Z_AB-avg_Z_A*avg_Z_B

In [19]:
f_1 = 0.5*(((avg_Z_full-(avg_Z_A*avg_Z_BC))**2 + (avg_Z_full-(avg_Z_B*avg_Z_AC))**2)/(2*3*Z_norm**2)-2)
f_2 = (avg_Z_full-avg_Z_A*avg_Z_B*avg_Z_C)**2/(2*3*Z_norm**2)-2
f_3 = (C_AB)**2/(2*2*Z_norm**2) 
f_upper = 0.5*(1+C_AB**2/(Var_A*Var_B)+2*np.log(4))


print(" f_1 = ", np.real(f_1),"\n f_2 = ", np.real(f_2),"\n f_3 =",np.real(f_3),"\n f_4 = ", np.real(f_upper))

 f_1 =  -0.9913636641543664 
 f_2 =  -1.993910272068639 
 f_3 = 4.90333318349294e-06 
 f_4 =  1.8863046520502942
